In [2]:
import numpy as np
def compactState(yard):
    sort = []
    for stack in yard:
      for container in stack:
        if not container in sort:
          sort.append(container)
    sort = sorted(sort)
    maxValue = 0
    for i in range(len(yard)):
      for j in range(len(yard[i])):
        yard[i][j] = sort.index(yard[i][j]) + 1
        if yard[i][j] > maxValue:
          maxValue = yard[i][j]
    return yard

def elevateState(yard, h, max_item):
    for stack in yard:
      while len(stack) < h:
        stack.insert(0,2*max_item)
    return yard

def flattenState(state):
    flatten = []
    for lista in state:
        for item in lista:
            flatten.append(item)
    return flatten

def normalize(state,max_item):
    return np.array(state)/max_item


def getStackValues(yard): #sorted stacks?
    values = []
    for stack in yard:
        flag = False
        cont = 0
        for i in range(len(stack)):
            if i==0:
                cont += 1
            else:
                if stack[i] <= stack[i-1]:
                    cont += 1
                else: break
        values.append(cont)
    return values

def getStackLen(yard):
    lens = []
    for stack in yard:
        lens.append(len(stack))
    return lens

def getTopStacks(yard,max_item):
    tops = []
    for stack in yard:
        if len(stack) != 0:
            tops.append(stack[len(stack)-1])
        else:
            tops.append(max_item)
    return tops

In [3]:
import copy
def generate_ann_state(yard,H):
    yard=copy.deepcopy(yard)
    max_item = max(set().union(*yard))
    #max_item = max([max(stack) for stack in yard])
    stackValues=getStackValues(yard) #well-placed
    stacksLen = getStackLen(yard)
    topStacks = getTopStacks(yard,max_item)
    yard=compactState(yard)
    yard=elevateState(yard,H, max_item)
    yard=flattenState(yard)
    yard=normalize(yard,max_item)
    state=yard
    #state=np.append(state,normalize(stackValues,max_item))
    #state=np.append(state,normalize(stacksLen,max_item))
    #state=np.append(state,normalize(topStacks,max_item))
    return state

In [4]:
import copy
import Greedy as gr
import Layout

def generate_ann_state_stack(layout, s, H):
    layout=copy.deepcopy(layout)
    max_item=max(set().union(*layout.stacks))
    
    #SF variables
    sf_possible = gr.SF_move_(layout)[0]!=False
    is_sorted = layout.is_sorted_stack(s)
    
    min_dif = 2*max_item
    for i in range(len(layout.stacks)):
        if i==s or len(layout.stacks[s])==0: continue
        c = layout.stacks[s][-1]
        dif = Layout.gvalue(layout.stacks[i]) - c
        
        if layout.is_sorted_stack(i) and len(layout.stacks[i]) < H and (dif >= 0) and (dif < min_dif):
            min_dif=dif    
    min_dif/=max_item     
    top = Layout.gvalue(layout.stacks[s])/max_item
    
    #SD variables
    size = len(layout.stacks[s])/H
    if size==0: mean = 0
    else: mean = np.mean(layout.stacks[s])/max_item
    if not is_sorted: rh=0
    else: rh = Layout.reachable_height(layout, s)/H
    sd_active = len(layout.moves)>0  and (layout.prev!="bg" and layout.moves[-1][0]==s)
    sorted_elements = layout.sorted_elements[s]/H
    
    return np.array([sf_possible,is_sorted,min_dif,top,size,mean,rh,sd_active, sorted_elements])


In [7]:
import Pixie as px
layout = px.read("Instancias/CVS/5-5/data5-5-1.dat")
gr.greedy_solve(layout,25)
print(layout.stacks)
for i in range(5):
    state=generate_ann_state_stack(layout, i, 7)
    print(state)
#state.shape=(5,7)
#state

[[22, 20, 14, 12, 6, 2, 16], [21, 19, 11, 9, 8, 5, 1], [7, 24, 3, 17, 18, 15], [], [25, 4, 13, 10, 23]]
[1.         0.         2.         0.64       1.         0.52571429
 0.         0.         0.85714286]
[1.         1.         2.         0.04       1.         0.42285714
 1.         0.         1.        ]
[1.         0.         2.         0.6        0.85714286 0.56
 0.         0.         0.14285714]
[1. 1. 2. 4. 0. 0. 1. 1. 0.]
[1.         0.         2.         0.92       0.71428571 0.6
 0.         0.         0.28571429]


In [8]:
import Greedy
lay = px.read("Instancias/CVS/5-5/data5-5-1.dat")
Greedy.greedy_solve(lay,20)
lay.stacks

[[22, 20, 14, 12],
 [21, 19, 11, 9, 8, 5, 1],
 [7, 24, 3, 17, 6],
 [15, 18, 16],
 [25, 4, 13, 10, 23, 2]]

In [9]:
from __future__ import print_function
import os
import neat
import visualize
import copy
from Greedy import greedy_solve
import Layout

layouts = []
layouts.append(px.read("Instancias/CVS/5-5/data5-5-1.dat"))
layouts.append(px.read("Instancias/CVS/5-5/data5-5-2.dat"))
layouts.append(px.read("Instancias/CVS/5-5/data5-5-3.dat"))
layouts.append(px.read("Instancias/CVS/5-5/data5-5-4.dat"))
layouts.append(px.read("Instancias/CVS/5-5/data5-5-6.dat"))


def generate_initial_layouts():
    global layouts
    return copy.deepcopy(layouts)

def eval_genome(genome, config):
    net = neat.nn.FeedForwardNetwork.create(genome, config)
    layouts=generate_initial_layouts()

    genome.fitness=0.0

    for layout in layouts: 
        best_ev=-1.0
        lay_copy = copy.deepcopy(layout)
        max_steps = greedy_solve(lay_copy, max_steps=100)

        for i in range(int(1.2*max_steps)):
            max_value=-np.inf
            for s in range(len(layout.stacks)):
                output = net.activate(generate_ann_state_stack(layout, s, 7))
                value = output[0]
                if value>max_value: 
                    max_value=value
                    s_o=s

            s_d,_=Layout.select_destination_stack(layout, s_o)
            if s_d==None: break 
            layout.move(s_o,s_d)

            lay_copy = copy.deepcopy(layout)
            steps = greedy_solve(lay_copy, max_steps=1.2*max_steps)
            if steps == None: break

            ev = -float(steps)/float(max_steps) + 0.01*i
            if ev > best_ev: best_ev=ev
        genome.fitness+=best_ev

    genome.fitness/=5.0
    return genome.fitness

def eval_genomes(genomes, config):
    
    for genome_id, genome in genomes:
        eval_genome(genome, config)
        

In [12]:
"""
Runs evaluation functions in parallel subprocesses
in order to evaluate multiple genomes at once.
"""
from multiprocessing import Pool

class ParallelEvaluator(object):
    def __init__(self, num_workers, eval_function, timeout=None):
        """
        eval_function should take one argument, a tuple of
        (genome object, config object), and return
        a single float (the genome's fitness).
        """
        self.num_workers = num_workers
        self.eval_function = eval_function
        self.timeout = timeout
        self.pool = Pool(num_workers)

    def __del__(self):
        self.pool.close() # should this be terminate?
        self.pool.join()


    def evaluate(self, genomes, config):
        jobs = []
        for ignored_genome_id, genome in genomes:
            jobs.append(self.pool.apply_async(self.eval_function, (genome, config)))

        # assign the fitness back to each genome
        for job, (ignored_genome_id, genome) in zip(jobs, genomes):
            genome.fitness = job.get(timeout=self.timeout)
            
evaluator = ParallelEvaluator(15, eval_genome)

In [ ]:
from __future__ import print_function
import os
import neat
import visualize



def run(config_file):
    # Load configuration.
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(50))

    # Run for up to 300 generations.
    winner = p.run(evaluator.evaluate, 3000)

    # Display the winning genome.
    print('\nBest genome:\n{!s}'.format(winner))

    #node_names = {-1:'A', -2: 'B', 0:'A XOR B'}
    #visualize.draw_net(config, winner, True, node_names=node_names)
    #visualize.plot_stats(stats, ylog=False, view=True)
    #visualize.plot_species(stats, view=True)

    p = neat.Checkpointer.restore_checkpoint('neat-checkpoint-4')
    p.run(eval_genomes, 10)



# Determine path to configuration file. This path manipulation is
# here so that the script will run successfully regardless of the
# current working directory.
local_dir = os.path.dirname(".")
config_path = os.path.join(local_dir, 'neat-config.txt')
run(config_path)


 ****** Running generation 0 ****** 



In [12]:
p = neat.Checkpointer.restore_checkpoint('neat-checkpoint-1661')


In [13]:
eval_genomes(p.population.items(), p.config)

In [14]:
max_ev=-1.0
for gen in p.population:
    if p.population[gen].fitness > max_ev: 
        max_ev=p.population[gen].fitness
        genome=p.population[gen]
max_ev

-0.5593837535014006

In [25]:
evals=np.zeros(40)
evalss = []
for k in range(1,41):
    layout = px.read("Instancias/CVS/5-10/data5-10-"+str(k)+".dat")
    
    #print(genome.fitness)
    net = neat.nn.FeedForwardNetwork.create(genome, p.config)

    lay_copy = copy.deepcopy(layout)
    max_steps = greedy_solve(lay_copy, max_steps=100)
    if max_steps==None: max_steps=100 
    min_steps = max_steps
    best_i=-1
    #print(max_steps)
    for i in range(max_steps):  
        max_value=-np.inf
        vals = []
        for s in range(len(layout.stacks)):
            output = net.activate(generate_ann_state_stack(layout, s, 7))
            value = output[0]
            vals.append(value)
            if value>max_value: 
                max_value=value
                s_o=s

        s_d,_=Layout.select_destination_stack(layout, s_o)
        if s_d==None: continue 
        layout.move(s_o,s_d)

        lay_copy = copy.deepcopy(layout)
        steps = greedy_solve(lay_copy, max_steps=1.2*max_steps)
        if steps==None: continue
        if layout.unsorted_stacks==0:
            print("solved!", layout.steps)
            break
        #print(vals, "\n", layout.stacks, i+1, steps)
        #print(layout.stacks,steps)
        if steps<min_steps:
            min_steps=steps

        if steps>1.2*max_steps: break
  

    print(max_steps,min_steps)
    
    if max_steps<100: evalss.append(max_steps)
    evals[k-1]=min_steps
print(np.mean(evals) )
print(np.mean(evalss) )

54 54
54 53
solved! 53
55 53
solved! 54
58 52
solved! 59
59 51
solved! 48
50 47
solved! 55
59 55
solved! 56
58 53
49 47
58 55
56 51
solved! 49
49 48
solved! 53
53 52
57 54
51 51
solved! 57
61 54
61 60
solved! 44
44 42
56 50
solved! 49
53 49
48 48
56 53
solved! 45
50 44
solved! 43
48 43
50 49
solved! 49
51 48
solved! 56
57 55
55 55
solved! 54
57 53
solved! 54
55 53
54 53
44 41
51 51
solved! 50
52 49
54 49
solved! 47
51 47
solved! 47
49 47
56 54
54 54
solved! 39
41 39
50.4
53.2


![image](https://i.imgur.com/s7kq3Md.png)